In [1]:
%matplotlib inline
import pandas as pd
from pipeline import * 

/Users/jackie16201/opt/anaconda3/envs/torch_environment/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jackie16201/Desktop/Spring_2023/fragments_discovery_design_ML/src/pipeline.py:11: DeprecationWarning: The rdkit.Chem.MCS module is deprecated; please use rdkit.Chem.rdFMCS instead.
  from rdkit.Chem import MCS


In [11]:
# ACTUAL VALUES for NG round 1

# values for processing fragments and compounds
fragment_path = '../../melis_gonorrhea/out/processed_predictions_with_new_bayHOv211162022_model/enamine_frags_18mil_with_FINALbayHO11152022_melis_predictions_11_18_2022.csv'
compound_path = '../out/test_800k.csv'
result_path = '../out/test_fragment_algo/'
fragment_smi_col = 'smiles'
compound_smi_col = 'smiles'
fragment_hit_col = 'hit'
compound_hit_col = 'hit'
cpd_name_col = 'Name'

# filters and thresholds for fragments and compounds
fragment_score = 0.2
compound_score = 0.3
fragment_remove_pains_brenk = 'both' # one of 'both', 'pains', 'brenk', 'none'
compound_remove_pains_brenk = 'both' # one of 'both', 'pains', 'brenk', 'none'
fragment_druglikeness_filter = [] # list containing 'egan', 'ghose', 'lipinski', 'muegge'
compound_druglikeness_filter = [] # list containing 'egan', 'ghose', 'lipinski', 'muegge'
fragment_require_more_than_coh = True
nitrofuran = 'O=[N+](O)c1ccco1'
sulfonamide = 'NS(=O)=O'
quinolone = 'O=c1cc[nH]c2ccccc12'
fragment_remove_patterns = [nitrofuran, sulfonamide, quinolone]

# input for matching and comparison to existing datasets
frags_cannot_disrupt_rings = True
display_inline_candidates = True
analogues_pval_diff_thresh = 0.05
analogues_absolute_diff_thresh = 0

# toxicity
toxicity_threshold_if_present = 0.5

# antibiotics
abx_path = '../../melis_gonorrhea/data/04052022_CLEANED_v5_antibiotics_across_many_classes.csv'
abx_smiles_col = 'Smiles'
abx_name_col = 'Name'
cpd_sim_to_abx = 0.5

# training set
train_set_path = '../../melis_gonorrhea/data/FULL_10_26_2022.csv'
train_set_smiles_col = 'SMILES'
train_set_name_col = 'Name'
cpd_sim_to_train_set = 0.5

# purchasable libraries
purch_path = '../../generativeML/data/Broad_800K_purchasable.xlsx'
purch_name_col = 'BROADID'
purch_name_needs_split = True

# tested before libraries - can be more expansive than train set
tested_before_path = '../../melis_gonorrhea/data/FULL_10_26_2022.csv'
tested_before_name_col = 'Name'
tested_before_name_needs_split = False

In [3]:
def run_pipeline(fragment_path, compound_path, result_path, fragment_smi_col = 'smiles', compound_smi_col = 'smiles', fragment_hit_col = 'hit', compound_hit_col = 'hit', fragment_score = 0.2, compound_score = 0.2, fragment_require_more_than_coh = True, fragment_remove_pains_brenk = 'both', compound_remove_pains_brenk = 'both', fragment_druglikeness_filter = [], compound_druglikeness_filter = [], fragment_remove_patterns = [], frags_cannot_disrupt_rings = True, toxicity_threshold_if_present = 0, abx_path = '', abx_smiles_col = 'smiles', abx_name_col = 'Name', train_set_path = '', train_set_smiles_col = 'smiles', train_set_name_col = 'Name', analogues_pval_diff_thresh = 0, analogues_absolute_diff_thresh = 0, cpd_name_col = 'Name', display_inline_candidates=False, purch_path='', purch_name_col='Name', purch_name_needs_split=False, tested_before_path='', tested_before_name_col='Name', tested_before_name_needs_split=False, cpd_sim_to_abx=0, cpd_sim_to_train_set=0):

    ##### part 1: process frags and compounds #####
    print('\nProcessing fragments...')
    df, mols, _ = process_dataset(frag_or_cpd='frag', path=fragment_path, score=fragment_score, smi_col=fragment_smi_col, hit_col=fragment_hit_col, require_more_than_coh=fragment_require_more_than_coh, remove_pains_brenk=fragment_remove_pains_brenk, remove_patterns=fragment_remove_patterns, druglikeness_filter=fragment_druglikeness_filter)
    print('\nProcessing compounds...')
    cpd_df, cpd_mols, full_cpd_df = process_dataset(frag_or_cpd='cpd', path=compound_path, score=compound_score, smi_col=compound_smi_col, hit_col=compound_hit_col, require_more_than_coh=False, remove_pains_brenk=compound_remove_pains_brenk, remove_patterns=[], druglikeness_filter=compound_druglikeness_filter)
    print('\nMatching fragments in compounds...')
        
    ##### part 2: get all matching frag / molecule pairs #####
    frag_match_indices, cpd_match_indices_lists = match_frags_and_mols(mols, cpd_mols)
    if frags_cannot_disrupt_rings:
        # for all matching fragments, keep only matches that do not disrupt rings
        frag_match_indices, cpd_match_indices_lists = check_for_complete_ring_fragments(mols, frag_match_indices, cpd_mols, cpd_match_indices_lists)
    rank_df = compile_results_into_df(df, cpd_df, mols, frag_match_indices, cpd_match_indices_lists, result_path)

    ##### part 3: add additional data and filters #####
    # check for frags associated with toxicity in the in-house tox database
    rank_df = filter_out_toxicity(rank_df, toxicity_threshold_if_present, 'matched_fragments', mols)
    # check for frags within abx or cpds close to known abx - does not remove any cpds
    rank_df, abx_mols, abx_names = check_abx(abx_path, abx_smiles_col, abx_name_col, rank_df, 'matched_fragments', mols, 'matched_molecules', cpd_mols)
    # check for frags within train set or molecules close to train set - again does not remove cpds
    rank_df, ts_mols, ts_names = check_training_set(train_set_path, train_set_smiles_col, train_set_name_col, rank_df, 'matched_fragments', mols, 'matched_molecules', cpd_mols)

    ##### part 4: statistical significance on analogues test #####
    if analogues_pval_diff_thresh > 0 or analogues_absolute_diff_thresh > 0: 
        print('Checking analogues of compounds with and without fragments...')
        rank_df = find_cpds_with_and_without_frags_for_whole_list(rank_df, full_cpd_df, compound_smi_col, compound_hit_col, 'matched_fragments', mols, 'matched_molecules', cpd_mols)
        rank_df = threshold_on_stat_sign_analogues_with_and_without_frags(rank_df, pval_diff_thresh=analogues_pval_diff_thresh, absolute_diff_thresh=analogues_absolute_diff_thresh)

    ##### part 5: visualization #####
    # save rank_df
    rank_df = rank_df.sort_values('number_of_matched_molecules', ascending = False)
    rank_df.to_csv(result_path + 'candidates_after_matching_and_filtering.csv', index = False)

    # group fragments and see clusters
    frag_folder = result_path + 'fragment_clusters/'
    os.mkdir(frag_folder)
    fragment_mols_for_plotting = add_legends_to_fragments(rank_df, smiles_column = 'fragment_SMILES')
    rank_df = extract_legends_and_plot(rank_df, fragment_mols_for_plotting, plot_suffix='cluster.png', path=frag_folder, murcko_scaffold=False, num_clusters = int(len(rank_df)/5))
    rank_df.to_csv(frag_folder + 'finalmols.csv', index = False)
        
    # draw all fragments
    draw_mols([mols[i] for i in list(rank_df['matched_fragments'])], [str(i) for i in range(0, len(rank_df))], result_path + 'ALL_FRAGMENT_MOLS.png', cut_down_size = False)

    # look at molecules corresponding to fragments
    cpd_names = list(cpd_df[cpd_name_col])
    if display_inline_candidates:
        candidate_folder = result_path + 'candidate_info/'
        os.mkdir(candidate_folder)
        plot_final_fragments_with_all_info(rank_df, candidate_folder, mols, cpd_mols, cpd_names, abx_mols, abx_names, ts_mols, ts_names)        

    ##### part 6: save relevant information #####
    # get the final matching molecules for saving
    all_matching_mol_indices = [x for l in list(rank_df['matched_molecules']) for x in l]
    all_matching_mol_indices = list(set(all_matching_mol_indices)) # deduplicate
    print('final number of molecules to test: ', len(all_matching_mol_indices))

    # save the names
    all_matching_mols = [cpd_names[i] for i in list(set(all_matching_mol_indices))]
    cpd_smiles = list(cpd_df[compound_smi_col])
    all_matching_smis = [cpd_smiles[i] for i in list(set(all_matching_mol_indices))]

    # and save the final molecules to df
    all_matching_mols_df = pd.DataFrame()
    all_matching_mols_df[cpd_name_col] = all_matching_mols
    all_matching_mols_df[compound_smi_col] = all_matching_smis

    # add metadata to mols
    cpd_df_meta = cpd_df[[cpd_name_col, compound_hit_col]]
    all_matching_mols_df = all_matching_mols_df.merge(cpd_df_meta, on = cpd_name_col)
    all_matching_mols_df.to_csv(result_path + 'candidate_compounds_after_matching_and_filtering_with_metadata.csv')

    ##### part 7: additional filtering #####
    # keep only molecules IN the PURCHASABLE Broad800K library
    all_matching_mols_df = filter_for_existing_mols(all_matching_mols_df, cpd_name_col, looking_for_presence=True, test_path=purch_path, test_name_col=purch_name_col, test_name_needs_split=purch_name_needs_split)
    print('length of df with purchasable mols: ', len(all_matching_mols_df))
    # make only molecules NOT IN previously-tested dataframes (make sure none of the molecules are exact matches to those tested before)
    all_matching_mols_df = filter_for_existing_mols(all_matching_mols_df, cpd_name_col, looking_for_presence=False, test_path=tested_before_path, test_name_col=tested_before_name_col, test_name_needs_split=tested_before_name_needs_split)
    print('length of df that has not been tested before: ', len(all_matching_mols_df))

    # now do tanimoto filtering on antibiotics and training set
    current_all_matching_mols = [Chem.MolFromSmiles(smi) for smi in list(all_matching_mols_df[compound_smi_col])]
    all_matching_mols_df['tan to nearest abx'] = for_mol_list_get_lowest_tanimoto_to_closest_mol(current_all_matching_mols, abx_mols)
    all_matching_mols_df['tan to nearest ts'] = for_mol_list_get_lowest_tanimoto_to_closest_mol(current_all_matching_mols, ts_mols)

    if cpd_sim_to_abx > 0:
        all_matching_mols_df = all_matching_mols_df[all_matching_mols_df['tan to nearest abx'] < cpd_sim_to_abx]
        print('length of all preds with tan abx < ' + str(cpd_sim_to_abx) + ': ', len(all_matching_mols_df))
    if cpd_sim_to_train_set > 0:
        all_matching_mols_df = all_matching_mols_df[all_matching_mols_df['tan to nearest ts'] < cpd_sim_to_train_set]
        print('length of all preds with tan ts < ' + str(cpd_sim_to_train_set) + ': ', len(all_matching_mols_df))

    ##### part 8: final round of visualization #####
    # cluster molecules based on fragment they have
    frags = [Chem.MolFromSmiles(smi) for smi in list(rank_df['fragment_SMILES'])]   
    all_matching_mols_df = cluster_mols_based_on_fragments(all_matching_mols_df, compound_smi_col, cpd_name_col, frags, result_path)
    rank_df.to_csv(result_path + 'FINAL_fragments_with_metadata.csv', index = False)
        
    # group fragments and see clusters
    final_folder = result_path + 'FINAL_mols_after_all_thresholding/'
    os.mkdir(final_folder)
    mols = add_legends_to_compounds(all_matching_mols_df, smiles_column = compound_smi_col, name_column = cpd_name_col)
    df = extract_legends_and_plot(all_matching_mols_df, mols, 'cluster.png', path=final_folder, murcko_scaffold=True, num_clusters = int(len(all_matching_mols_df)/5))
    all_matching_mols_df.to_csv(final_folder + 'final_proposed_molecules_to_order.csv')

    return(all_matching_mols_df)


Processing fragments...
length of df:  18338026
length of df >0.4:  2155
length of df with more than C,O,H characters:  2155
length of df with valid mols:  2155
length of all preds with clean (no PAINS or Brenk) mols:  2155


/Users/jackie16201/Desktop/Spring_2023/fragments_discovery_design_ML/src/pipeline.py:101: DeprecationWarning: The rdkit.Chem.MCS module is deprecated; please use rdkit.Chem.rdFMCS instead.
  mcs = MCS.FindMCS([mol, pattern_mol], atomCompare='elements',completeRingsOnly = True)


length of df with no O=[N+](O)c1ccco1:  2155
length of df with no NS(=O)=O:  1648
length of df with no O=c1cc[nH]c2ccccc12:  1626

Processing compounds...
length of df:  798200
length of df >0.5:  8496
length of df with valid mols:  8496
length of all preds with clean (no PAINS or Brenk) mols:  1080

Matching fragments in compounds...
number of matched fragments:  6
Previewing dataframe so far...


,matched_fragments,fragment_SMILES,length_of_fragment,matched_molecules,number_of_matched_molecules,fragment_scores,full_molecule_scores,average_molecule_score
2,752,NC(=O)C1=NC=CC(Cl)=C1Cl,11,"[159, 294, 365, 603]",4,0.404153,"[0.5057420745491982, 0.6340445172786713, 0.590...",0.602128
0,5,FC1=C(N2CCOCC2)C=CC2=C(Cl)C=CN=C12,18,[963],1,0.669166,[0.5164243218302726],0.516424
1,412,O=C1C=CC=NN1CC1=CC=C(Cl)C(Cl)=C1,16,[492],1,0.410167,[0.5003664669394493],0.500366
3,997,O=C(NC1=CC=C(Cl)C(Cl)=C1)N1CCOCC1,17,[977],1,0.477369,[0.5486092334985733],0.548609
4,1564,CN1C=CN=C1NC(=O)C1=CC=C(Cl)C(Cl)=C1,17,[790],1,0.456231,[0.5006607256829738],0.500661


In [ ]:
run_pipeline(fragment_path=fragment_path, compound_path=compound_path, result_path=result_path, fragment_smi_col=fragment_smi_col, compound_smi_col=compound_smi_col, fragment_hit_col=fragment_hit_col, compound_hit_col=compound_hit_col, fragment_score=fragment_score, compound_score=compound_score, fragment_require_more_than_coh=fragment_require_more_than_coh, fragment_remove_pains_brenk=fragment_remove_pains_brenk, compound_remove_pains_brenk=compound_remove_pains_brenk, fragment_druglikeness_filter=fragment_druglikeness_filter, compound_druglikeness_filter=compound_druglikeness_filter, fragment_remove_patterns=fragment_remove_patterns, frags_cannot_disrupt_rings=frags_cannot_disrupt_rings, abx_path=abx_path, abx_smiles_col=abx_smiles_col, abx_name_col=abx_name_col, train_set_path=train_set_path, train_set_smiles_col=train_set_smiles_col, train_set_name_col=train_set_name_col, analogues_pval_diff_thresh=analogues_pval_diff_thresh, analogues_absolute_diff_thresh=analogues_absolute_diff_thresh, cpd_name_col=cpd_name_col, display_inline_candidates=display_inline_candidates, purch_path=purch_path, purch_name_col=purch_name_col, purch_name_needs_split=purch_name_needs_split, cpd_sim_to_abx=cpd_sim_to_abx, cpd_sim_to_train_set=cpd_sim_to_train_set)

In [ ]:
# # broad800k mols
# broad800k = pd.read_csv('../../melis_gonorrhea/out/processed_predictions_with_new_bayHOv211162022_model/broad800K_melis_predictions_with_FINALbayHO11152022_11_16_2022.csv')
# metadata = pd.read_csv('../../nontoxic_stat_phase_killing_abx/data/PublicStructures.txt', sep = '\t')
# broad800k = broad800k.merge(metadata, left_on = 'smiles', right_on = 'SMILES', how = 'left')
# broad800k = broad800k.drop_duplicates('smiles')
# broad800k = broad800k[['smiles', 'Name', 'hit', 'hit_epi_unc']]
# broad800k.to_csv('../out/test_800k.csv')

In [ ]:
# meta = pd.read_csv('../out/processed_predictions_with_new_bayHOv211162022_model/enamine_frags_18mil_with_FINALbayHO11152022_melis_predictions_11_18_2022.csv')
# meta = meta[['smiles', 'idnumber', 'Type']]
# rank_df = rank_df.merge(meta, left_on = 'fragment_SMILES', right_on = 'smiles')
# display(rank_df)